<a href="https://colab.research.google.com/github/dbengp/dio-ml/blob/main/transfer_learning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
import keras
import matplotlib.pyplot as plt

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from matplotlib.pyplot import imshow

In [32]:
root = 'PetImages'
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root)]]

print(categories)

['PetImages/Dog', 'PetImages/Cat', 'PetImages/.ipynb_checkpoints']


In [9]:
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [10]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

num_classes = len(categories)

In [11]:
random.shuffle(data)

In [12]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [13]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1]


In [14]:
# normaliza o conjunto de dados
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# converte os rotulos por vetores
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

(301, 3)


In [15]:
# criação da rede, ao final tem-se o resumo da mesma
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

Input dimensions:  (224, 224, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# treinar nosso modelo para 10 épocas com um tamanho de lote de 128.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 115s 10s/step - accuracy: 0.4610 - loss: 0.8592 - val_accuracy: 0.5967 - val_loss: 0.6891
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 136s 10s/step - accuracy: 0.5565 - loss: 0.6896 - val_accuracy: 0.5933 - val_loss: 0.6782
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 140s 9s/step - accuracy: 0.6085 - loss: 0.6624 - val_accuracy: 0.5533 - val_loss: 0.6899
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 145s 10s/step - accuracy: 0.6275 - loss: 0.6451 - val_accuracy: 0.6000 - val_loss: 0.6441
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 111s 10s/step - accuracy: 0.6691 - loss: 0.6164 - val_accuracy: 0.6400 - val_loss: 0.6148
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 146s 10s/step - accuracy: 0.6832 - loss: 0.5892 - val_accuracy: 0.6333 - val_loss: 0.6248
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 136s 10s/step - accuracy: 0.6892 - loss: 0.5636 - val_accuracy: 0.6933 - val_loss: 0.6014
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 142s 10s/step - accuracy: 0.7380 - loss: 0.5306 - val_accuracy: 0.6

In [17]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [18]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

In [19]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [20]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1106s 102s/step - accuracy: 0.5499 - loss: 0.7563 - val_accuracy: 0.7633 - val_loss: 0.5062
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1157s 102s/step - accuracy: 0.8112 - loss: 0.4512 - val_accuracy: 0.8367 - val_loss: 0.3935
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1085s 100s/step - accuracy: 0.8613 - loss: 0.3712 - val_accuracy: 0.8633 - val_loss: 0.3536
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1127s 102s/step - accuracy: 0.8487 - loss: 0.3552 - val_accuracy: 0.8567 - val_loss: 0.3333
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1165s 103s/step - accuracy: 0.8724 - loss: 0.3145 - val_accuracy: 0.8733 - val_loss: 0.3128
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1157s 102s/step - accuracy: 0.8810 - loss: 0.2877 - val_accuracy: 0.8533 - val_loss: 0.3140
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1144s 101s/step - accuracy: 0.8873 - loss: 0.2888 - val_accuracy: 0.8833 - val_loss: 0.3037
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1075s 98s/step - accuracy: 0.9003 - loss: 0.2752 - v

In [21]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.2459876984357834
Test accuracy: 0.9069767594337463


In [30]:
# para obter as prob de uma nova imagem.
img, x = get_image('new_data_for_predict/cachorro_inteligente.jpg')
probabilities = model_new.predict([x])
print(probabilities)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step
[[1.0000000e+00 2.0725719e-15 4.1004984e-14]]


In [31]:
img, x = get_image('new_data_for_predict/gato_inteligente.jpg')
probabilities = model_new.predict([x])
print(probabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
[[1.00000000e+00 9.01194813e-11 1.10136475e-11]]
